# Análisis Exploratorio de los Datasets Procesados (MEPS 2022)
Este notebook explora las variables principales de los archivos procesados en la carpeta `data/`. Se realiza un análisis estadístico y de distribución para variables demográficas, de salud, primas, pagos y empleo, considerando el tratamiento adecuado de datos faltantes o no aplicables.

In [16]:
# Librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [17]:
# Cargar los datasets procesados con rutas correctas
fyc = pd.read_csv('../data/meps_fyc_2022_processed.csv')
cond = pd.read_csv('../data/meps_cond_2022_processed.csv')
jobs = pd.read_csv('../data/meps_jobs_2022_processed.csv')
prpl = pd.read_csv('../data/meps_prpl_2022_processed.csv')

C:\Users\DJSix\AppData\Local\Temp\ipykernel_242220\1779530984.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  cond = pd.read_csv('../data/meps_cond_2022_processed.csv')


## Análisis General de Personas
- Cantidad de personas individuales
- Estadísticas de edad (mínima, máxima, Q1, Q3, media, mediana)
- Cantidad de personas por raza, estado civil y región
- Cantidad de personas en cada categoría de pobreza
        

In [18]:
# Detectar nombres de columnas relevantes de forma flexible (ignorando mayúsculas/minúsculas)
print('Columnas disponibles en fyc:', list(fyc.columns))

def buscar_columna(df, posibles):
    cols = [c for c in df.columns for p in posibles if c.lower() == p.lower()]
    return cols[0] if cols else None

col_age = buscar_columna(fyc, ['age', 'edad'])
col_race = buscar_columna(fyc, ['race', 'raza'])
col_marital = buscar_columna(fyc, ['marital_status', 'estado_civil'])
col_region = buscar_columna(fyc, ['region', 'región'])
col_poverty = buscar_columna(fyc, ['poverty_category', 'categoria_pobreza'])

# Análisis de personas en fyc
personas = fyc.drop_duplicates(subset='person_unique_id')

# Estadísticas de edad
edad = pd.to_numeric(personas['age_last_birthday'], errors='coerce')
print('Edad - min:', edad.min(), 'max:', edad.max(), 'Q1:', edad.quantile(0.25), 'Q3:', edad.quantile(0.75), 'media:', edad.mean(), 'mediana:', edad.median())

# Cantidad por raza
print('Cantidad por raza:')
print(personas['race_ethnicity'].value_counts(dropna=False))

# Cantidad por estado civil
print('Cantidad por estado civil:')
print(personas['marital_status_2022'].value_counts(dropna=False))

# Cantidad por región
print('Cantidad por región:')
print(personas['region_2022'].value_counts(dropna=False))

# Cantidad por categoría de pobreza
print('Cantidad por categoría de pobreza:')
print(personas['poverty_category_2022'].value_counts(dropna=False))

print('Cantidad de personas individuales:', len(personas))

Columnas disponibles en fyc: ['dwelling_unit_id', 'person_id', 'person_unique_id', 'panel_number', 'age_last_birthday', 'sex', 'race_ethnicity', 'marital_status_2022', 'region_2022', 'total_healthcare_exp_2022', 'total_out_of_pocket_exp_2022', 'poverty_category_2022', 'insurance_coverage_2022', 'perceived_health_status', 'person_weight_2022']
Edad - min: 0.0 max: 85.0 Q1: 23.0 Q3: 64.0 media: 43.56306005082252 mediana: 45.0
Cantidad por raza:
race_ethnicity
Non-Hispanic White only                  12211
Hispanic                                  4883
Non-Hispanic Black only                   3244
Non-Hispanic Asian only                   1220
Non-Hispanic Other race or multi-race      873
Name: count, dtype: int64
Cantidad por estado civil:
marital_status_2022
Married                      8602
Never married                5495
Under 16 - not applicable    3765
Divorced                     2546
Widowed                      1619
Separated                     397
-7                        

## Análisis de Condiciones Médicas
- Cantidad de condiciones médicas distintas
- Media de condiciones por persona
- Top 5 condiciones más comunes (general, mujeres, hombres)
        

In [ ]:
# Análisis de condiciones médicas en cond
print('Columnas disponibles en cond:', list(cond.columns))

# Usar directamente la columna correcta para condiciones médicas
col_condition = 'ccsr_category_1'

condiciones_distintas = cond[col_condition].nunique()
condiciones_por_persona = cond.groupby('person_unique_id')[col_condition].nunique()
print('Cantidad de condiciones médicas distintas:', condiciones_distintas)
print('Media de condiciones por persona:', condiciones_por_persona.mean())
print('Top 5 condiciones más comunes (general):')
print(cond[col_condition].value_counts().head(5))

Columnas disponibles en cond: ['person_unique_id', 'condition_id', 'panel_number', 'condition_round', 'age_at_diagnosis', 'injury_flag', 'icd10_code', 'ccsr_category_1']
Columna de condición médica no encontrada.


## Análisis de Primas y Pagos
- Estadísticas de prima out of pocket y otras columnas de pagos
        

In [20]:
# Primas y pagos: columnas relevantes en prpl
pagos = pd.to_numeric(prpl['out_of_pocket_premium_edited'], errors='coerce')
print('Estadísticas para out_of_pocket_premium_edited:')
print('min:', pagos.min(), 'max:', pagos.max(), 'media:', pagos.mean(), 'mediana:', pagos.median(), 'Q1:', pagos.quantile(0.25), 'Q3:', pagos.quantile(0.75))
print('Cantidad de valores válidos:', pagos.notna().sum())

Estadísticas para out_of_pocket_premium_edited:
min: 0.0 max: 4583.33 media: 306.679531166348 mediana: 212.5 Q1: 70.0 Q3: 433.33
Cantidad de valores válidos: 13075


## Análisis de Empleo y Seguro Médico
- Cantidad de personas con trabajo que ofrecen seguro médico
- Cantidad de personas por categoría de trabajo
- Estadísticas de horas trabajadas y salario por hora
        

In [21]:
# Empleo: columnas relevantes en jobs
trabajos = jobs.copy()
# Personas con trabajo que ofrecen seguro médico
ofrecen_seguro = trabajos[trabajos['insurance_offered'] == 1]['person_unique_id'].nunique()
print('Cantidad de personas con trabajo que ofrecen seguro médico:', ofrecen_seguro)
# Estadísticas de horas trabajadas y salario por hora
for col in ['hours_per_week', 'hourly_wage']:
    vals = pd.to_numeric(trabajos[col], errors='coerce')
    print(f'Estadísticas para {col}:')
    print('min:', vals.min(), 'max:', vals.max(), 'media:', vals.mean(), 'mediana:', vals.median(), 'Q1:', vals.quantile(0.25), 'Q3:', vals.quantile(0.75))
    print('Cantidad de valores válidos:', vals.notna().sum())

Cantidad de personas con trabajo que ofrecen seguro médico: 0
Estadísticas para hours_per_week:
min: 1.0 max: 168.0 media: 35.83322260975475 mediana: 40.0 Q1: 30.0 Q3: 40.0
Cantidad de valores válidos: 36126
Estadísticas para hourly_wage:
min: 0.0 max: 115.0 media: 20.648742049992602 mediana: 18.0 Q1: 15.0 Q3: 24.0
Cantidad de valores válidos: 13522
